In [19]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import datasets, metrics
from mealpy.bio_based import SMA #trocar por hho
import numpy as np

In [20]:
# Step 1: Load the CSV file
df = pd.read_csv('oriental_data.csv')
df

,time,dhi_pyr,ghi_pyr,air_temperature,relative_humidity,barometric_pressure,precipitation
0,2015-01-12 00:00:00,0.0,0.0,22.60,66.00,878.00,0.0
1,2015-01-12 00:01:00,0.0,0.0,22.30,71.25,877.50,0.0
2,2015-01-12 00:02:00,0.0,0.0,22.25,71.50,877.50,0.0
3,2015-01-12 00:03:00,0.0,0.0,22.20,71.60,877.50,0.0
4,2015-01-12 00:04:00,0.0,0.0,22.20,71.50,877.50,0.0
...,...,...,...,...,...,...,...
3070069,2022-11-30 23:56:00,NaN,0.0,26.61,100.00,999.91,0.0
3070070,2022-11-30 23:57:00,NaN,0.0,26.66,100.00,999.82,0.0
3070071,2022-11-30 23:58:00,NaN,0.0,26.69,100.00,999.91,0.0
3070072,2022-11-30 23:59:00,NaN,0.0,26.68,100.00,999.96,0.0


In [21]:
# Step 2: Preprocess the data
# Filter the necessary columns for input and target variables
df = df[['time', 'air_temperature', 'relative_humidity', 'dhi_pyr']]

In [22]:
# Drop rows with missing values in the relevant columns
df.dropna(subset=['air_temperature', 'relative_humidity', 'dhi_pyr'], inplace=True)

df.time = pd.to_datetime(df['time'])
df.time

clean_df = df[df.time.dt.strftime('%H:%M').between('06:00','19:00')]
df = clean_df
df


C:\Users\Giuli\AppData\Local\Temp\ipykernel_41420\2585636722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['air_temperature', 'relative_humidity', 'dhi_pyr'], inplace=True)
C:\Users\Giuli\AppData\Local\Temp\ipykernel_41420\2585636722.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time = pd.to_datetime(df['time'])


,time,air_temperature,relative_humidity,dhi_pyr
360,2015-01-12 06:00:00,20.000000,81.700000,24.200
361,2015-01-12 06:01:00,20.050000,81.700000,25.475
362,2015-01-12 06:02:00,20.100000,81.650000,26.875
363,2015-01-12 06:03:00,20.100000,81.500000,28.675
364,2015-01-12 06:04:00,20.100000,81.450000,30.325
...,...,...,...,...
1275531,2018-03-31 18:56:00,20.183333,90.533333,0.000
1275532,2018-03-31 18:57:00,20.150000,90.516667,0.000
1275533,2018-03-31 18:58:00,20.116667,90.716667,0.000
1275534,2018-03-31 18:59:00,20.105000,90.633333,0.000


In [23]:
# Step 3: Prepare the input features (X) and target variable (y)
X = df[['air_temperature', 'relative_humidity']]  # Features
y = df['dhi_pyr']  # Target



In [24]:
# Step 4: Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [26]:
KERNEL_ENCODER = LabelEncoder()
KERNEL_ENCODER.fit(['linear', 'poly', 'rbf', 'sigmoid'])

LabelEncoder()

In [27]:
# Step 5: Initialize and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_std, y_train.values.ravel())



RandomForestRegressor(random_state=42)

In [28]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
def fitness_function(solution, flg = None):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid

#     kernel_encoded = round(solution[0])
#     c = solution[1]
#     kernel_decoded = KERNEL_ENCODER.inverse_transform([kernel_encoded])[0]

#     svc = SVC(C=c, random_state=1, kernel=kernel_decoded, max_iter=100)
    rfr = RandomForestRegressor(max_depth = round(solution[1]), n_estimators=round(solution[0]))
    # Fit the model
#     svc.fit(X_train_std, y_train.values.ravel())
    # Make the predictions
#     y_predict = svc.predict(X_test_std)
    if(flg == 'run'):
        rfr.fit(X_train_std, y_train.values.ravel())
        return rfr
    # Measure the performance
#     x_print = cross_val_score(rfr, X_train_std, y_train.values.ravel(), cv=3).mean()
    accuracy = cross_validate(rfr, X_train_std, y_train.values.ravel(), cv=5, scoring='max_error', error_score='raise')
    result = np.array(accuracy['test_score']).mean()
    return abs(result)

problem = {
    "fit_func": fitness_function,
    "lb": [  1,  11],
    "ub": [500, 50],
    "minmax": "min",
}

In [29]:
fitness_function([20, 10])

np.float64(389.9138722080078)

In [30]:
from mealpy.swarm_based.SMA import BaseSMA
import mealpy.bio_based.SMA as SMA

model = SMA.BaseSMA(epoch=5, pop_size=20)
model.solve(problem)
print(f"Best solution: {model.solution[0]}")
# print(f"Best kernel: {KERNEL_ENCODER.inverse_transform([int(model.solution[0][0])])[0]}, Best c: {model.solution[0][1]}")

print(f"Best accuracy: {model.solution[1]}")

ModuleNotFoundError: No module named 'mealpy.swarm_based.SMA'

In [ ]:
melhor_modelo = fitness_function(model.solution[0], flg='run')
y_pred = melhor_modelo.predict(X_test_std)

In [ ]:
# import pylab as plt
import matplotlib.pyplot as plt

plt.figure(dpi=150)

plt.plot(y_test, y_test, 'k-')
plt.plot(y_test[::30], y_pred[::30], 'b.', linewidth=0.2)

plt.show()

In [ ]:
model.solution

In [ ]:
# Example to print or store metrics after running the model
import numpy as np

# Assuming `model.history` contains the best solution of each iteration (if available in Mealpy)
if hasattr(model, 'history'):
    best_fitness_per_iteration = model.history['fitness']
    mean_fitness = np.mean(best_fitness_per_iteration)
    std_fitness = np.std(best_fitness_per_iteration)
    
    print(f"Mean Fitness: {mean_fitness}")
    print(f"Standard Deviation of Fitness: {std_fitness}")
    print(f"Final Best Fitness: {model.solution[1][0]}")

    # Convergence curve
    import matplotlib.pyplot as plt
    plt.plot(best_fitness_per_iteration)
    plt.xlabel("Iteration")
    plt.ylabel("Best Fitness")
    plt.title("Convergence Curve")
    plt.show()


In [ ]:
# # Step 6: Make predictions on the test set
# y_pred = rf_model.predict(X_test_std)
# x_print = metrics.r2_score(y_test.values.ravel(), y_predict)
# print(x_print)


In [ ]:
# # Step 7: Evaluate the model
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"Mean Absolute Error (MAE): {mae:.2f}")
# print(f"R-squared (R2): {r2:.2f}")